https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

# Setup

In [18]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable

## Setup Langsmith for improved debug capabilities

In [19]:
import getpass
import os

In [20]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

········


## Connect to OpenAI

In [21]:
from langchain_openai import ChatOpenAI

In [22]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()



model = ChatOpenAI(model="gpt-4o")

········


In [8]:
?ChatOpenAI

## Directly pass messages to the language model

This is not the usual approach, but we start with this to demonstrate how it works.

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'stop', 'logprobs': None}, id='run-870f01d4-3a09-4f9d-bc9c-f91d6920c0b8-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [24]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [25]:
# Parse the AIMessage object to just extract the content text
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [26]:
# ...or, we can "chain" the operations together, using | symbol
# (option + shift + L to get pipe on azerty keyboard)

chain = model | parser

In [27]:
chain.invoke(messages)

'Ciao!'

## Use Prompt Templates

In [28]:
from langchain_core.prompts import ChatPromptTemplate

In [29]:
# First, let's create a string that we will format to be the system message
system_template = "Translate the following into {language}:"

In [30]:
# Next, we can create the PromptTemplate. This will be a combination of the system_template 
# as well as a simpler template for where the put the text.
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [31]:
# The input to this prompt template is a dictionary.
# We can play around with this prompt template by itself to see what it does by itself.
result = prompt_template.invoke({"language": "french", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:'), HumanMessage(content='hi')])

In [32]:
# We can see that it returns a ChatPromptValue that consists of two messages.
# If we want to access the messages directly we do:
result.to_messages()

[SystemMessage(content='Translate the following into french:'),
 HumanMessage(content='hi')]

In [33]:
# Now we can chain the components together as follows:
chain = prompt_template | model | parser

chain.invoke({
    "language": "mandarin",
    "text": "hi"
})

'你好 (nǐ hǎo)'